In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define Global Variable & Object

In [3]:
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

In [1]:
import numpy as np

def IC50_to_pIC50(ic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 9 - np.log10(ic50_values)

In [4]:
pIC50_to_IC50(10)

0.1

In [4]:
!pip install dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd

f = pd.read_csv('/content/drive/MyDrive/open/train.csv')
# s = pd.read_csv('/content/drive/MyDrive/open/SMILES_Big_Data_Set.csv')

f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18039 entries, 0 to 18038
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Molecule ChEMBL ID  1952 non-null   object 
 1   Standard Type       1952 non-null   object 
 2   Standard Relation   1952 non-null   object 
 3   Standard Value      1952 non-null   float64
 4   Standard Units      1952 non-null   object 
 5   pChEMBL Value       1952 non-null   float64
 6   Assay ChEMBL ID     1952 non-null   object 
 7   Target ChEMBL ID    1952 non-null   object 
 8   Target Name         1952 non-null   object 
 9   Target Organism     1952 non-null   object 
 10  Target Type         1952 non-null   object 
 11  Document ChEMBL ID  1952 non-null   object 
 12  IC50_nM             1952 non-null   float64
 13  pIC50               16989 non-null  float64
 14  Smiles              18039 non-null  object 
 15  mol                 16087 non-null  object 
 16  num_

In [ ]:
import numpy as np

f[f['pIC50']==0].count()

,0
Molecule ChEMBL ID,0
Standard Type,0
Standard Relation,0
Standard Value,0
Standard Units,0
pChEMBL Value,0
Assay ChEMBL ID,0
Target ChEMBL ID,0
Target Name,0
Target Organism,0


In [ ]:
f = f[f['pIC50']>0]

In [ ]:
# pIC50 열에서 결측치가 있는 행을 제거
f = f.dropna(subset=['pIC50'])

In [ ]:
f = f[f['pIC50']>=4]

In [ ]:
f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3863 entries, 0 to 3862
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Molecule ChEMBL ID  1952 non-null   object 
 1   Standard Type       1952 non-null   object 
 2   Standard Relation   1952 non-null   object 
 3   Standard Value      1952 non-null   float64
 4   Standard Units      1952 non-null   object 
 5   pChEMBL Value       1952 non-null   float64
 6   Assay ChEMBL ID     1952 non-null   object 
 7   Target ChEMBL ID    1952 non-null   object 
 8   Target Name         1952 non-null   object 
 9   Target Organism     1952 non-null   object 
 10  Target Type         1952 non-null   object 
 11  Document ChEMBL ID  1952 non-null   object 
 12  IC50_nM             1952 non-null   float64
 13  pIC50               3863 non-null   float64
 14  Smiles              3863 non-null   object 
 15  mol                 1911 non-null   object 
 16  num_atoms  

In [ ]:
fname = f"/content/drive/MyDrive/open/train.csv"
f.to_csv(fname, index=False)
print(f"'{fname}' saved complete.")

'/content/drive/MyDrive/open/train.csv' saved complete.


In [9]:
SUBMISSION_PATH = '/content/drive/MyDrive/case1/submit'

In [17]:
DATA_PATH = '/content/drive/MyDrive/case1/open'
SUBMISSION_PATH = '/content/drive/MyDrive/case1/submit'

from transformers import pipeline, AutoModelForMaskedLM

class ModelCfg:
    epochs = 5
    seed = 42
    n_fold = 5
    batch_size = 24

    # 모델명 https://huggingface.co/models 참고
    # Use a pipeline as a high-level helper
    # pipe = pipeline("fill-mask", model="seyonec/PubChem10M_SMILES_BPE_180k")
    MODEL_NAME  = 'seyonec/PubChem10M_SMILES_BPE_180k'
    # = 'seyonec/PubChem10M_SMILES_BPE_180k'
    MAX_SEQ_LENGTH = 0
    TOKENIZER = None
    MODEL_PATH = '/content/drive/MyDrive/case1/model'
    CLEAN_DATA_PATH = '/content/drive/MyDrive/case1/cleaned'
    CONFIG_PATH = MODEL_PATH + 'model_config.pth'

# Import Modules

In [18]:
import numpy as np
import pandas as pd
import random
import os
import dill
import matplotlib.pyplot as plt
import json
from ast import literal_eval
from tqdm.auto import tqdm
tqdm.pandas()
import re
import sys
import copy
import time
import math
import string
import pickle
import joblib
import itertools
import platform
import collections
import scipy as sp
import gc
import warnings; warnings.filterwarnings("ignore")
from scipy.stats import gmean

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

print('torch version : ' , torch.__version__)
print('tokenizers version : ', tokenizers.__version__)
print('transformers version : ', transformers.__version__)
%env TOKENIZERS_PARALLELISM=true


torch version :  2.4.1+cu121
tokenizers version :  0.19.1
transformers version :  4.44.2
env: TOKENIZERS_PARALLELISM=true


# Set Path

In [19]:
if not os.path.exists(ModelCfg.MODEL_PATH):
    print(f'create directory.....[{ModelCfg.MODEL_PATH}]')
    os.mkdir(ModelCfg.MODEL_PATH)
if not os.path.exists(ModelCfg.CLEAN_DATA_PATH):
    print(f'create directory.....[{ModelCfg.CLEAN_DATA_PATH}]')
    os.mkdir(ModelCfg.CLEAN_DATA_PATH)

# Set Processor

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device : ', device)

device :  cpu


# User Define Function

- Metric 함수

In [21]:
def MY_RMSELoss(yhat, y):
    return torch.sqrt(torch.mean((yhat-y)**2)) * 100

- 재현성 구현 함수

In [22]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

reset_seeds(ModelCfg.seed)

 # Load Data

In [23]:
def read_data_list():
    train = pd.read_csv(f'{DATA_PATH}/train_before.csv')
    test = pd.read_csv(f'{DATA_PATH}/test.csv')
    sample_submission = pd.read_csv(f'{DATA_PATH}/sample_submission.csv')

    print('train.shape: ', train.shape)
    print('test.shape: ', test.shape)
    print('sample_submission.shape: ', sample_submission.shape)

    return train, test, sample_submission

train, test, sample_submission = read_data_list()

train.shape:  (1952, 15)
test.shape:  (113, 2)
sample_submission.shape:  (113, 2)


# Create Tokenizer Object

In [24]:
from transformers import AutoTokenizer

def make_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained('seyonec/PubChem10M_SMILES_BPE_180k', normalization=True, caeche_dir='./cache')
    tokenizer.save_pretrained(f'{ModelCfg.MODEL_PATH}')
    print(f'tokenizer object load & save......[{ModelCfg.MODEL_NAME}]')

    example = "Cn1c(=O)c2c(ncn2C)n(C)c1=O"
    tokens = tokenizer(example) # example
    tokens_ = tokenizer.tokenize(example) # example
    print(tokens)
    print(tokens_)

    return tokenizer

tokenizer = make_tokenizer()
ModelCfg.TOKENIZER = tokenizer

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer object load & save......[seyonec/PubChem10M_SMILES_BPE_180k]
{'input_ids': [0, 315, 21, 71, 263, 51, 13, 71, 22, 71, 12, 358, 22, 39, 13, 82, 12, 39, 13, 71, 21, 33, 51, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Cn', '1', 'c', '(=', 'O', ')', 'c', '2', 'c', '(', 'ncn', '2', 'C', ')', 'n', '(', 'C', ')', 'c', '1', '=', 'O']


# Preparing Dataset for Torch

- max_length 계산

In [25]:
train

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
0,CHEMBL4443947,IC50,'=',0.022,nM,10.66,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,IC50,'=',0.026,nM,10.59,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,IC50,'=',0.078,nM,10.11,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,IC50,'=',0.081,nM,10.09,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,IC50,'=',0.099,nM,10.00,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,CHEMBL380009,IC50,'=',30000.000,nM,4.52,CHEMBL870864,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL1147720,30000.000,4.52,O=C(Nc1nc2cc[nH]cc-2n1)c1cccc([N+](=O)[O-])c1
1948,CHEMBL377654,IC50,'=',30000.000,nM,4.52,CHEMBL870864,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL1147720,30000.000,4.52,CCCCn1c(NC(=O)c2cccc(Cl)c2)nc2ccccc21
1949,CHEMBL208667,IC50,'=',30000.000,nM,4.52,CHEMBL870864,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL1147720,30000.000,4.52,O=C(Nc1nc2cc(F)c(F)cc2[nH]1)c1cccc([N+](=O)[O-...
1950,CHEMBL3403453,IC50,'=',42000.000,nM,4.38,CHEMBL3407004,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL3400143,42000.000,4.38,OC[C@H]1C[C@@H](Nc2nc(Nc3ccccc3)ncc2-c2nc3cccc...


In [26]:
def cacl_max_lenth(train_data):
    print(f'calculate max sequence length....[{ModelCfg.MODEL_NAME}]')
    for text_col in ['Smiles']:
        SMILES_lengths = []
        text_values = tqdm(train[text_col].fillna('').values, total=len(train))
        for text in text_values:
            length = len(ModelCfg.TOKENIZER(text, add_special_tokens=False)['input_ids'])
            SMILES_lengths.append(length)

    max_seq_length = max(SMILES_lengths) + 2
    print('SMILES max length : ', max(SMILES_lengths))
    print('final max data length : ', max_seq_length)

    return max_seq_length

max_seq_length = cacl_max_lenth(train)
ModelCfg.MAX_SEQ_LENGTH = max_seq_length

calculate max sequence length....[seyonec/PubChem10M_SMILES_BPE_180k]


  0%|          | 0/1952 [00:00<?, ?it/s]

SMILES max length :  102
final max data length :  104


- Dataset 준비

In [27]:
def prepare_input(tokenizer, max_seq_length, smiles):
    inputs = tokenizer(smiles, add_special_tokens=True, truncation=True, max_length=max_seq_length, padding="max_length", return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

def create_label(pIC50):
    return torch.tensor([pIC50], dtype=torch.float)

class ChemiDataset(Dataset):
    def __init__(self, cfg, df, is_train=False):
        self.cfg = cfg
        self.is_train = is_train
        self.smiles = df['Smiles'].values
        if not self.is_train:
            self.pIC50 = df['IC50_nM'].values
            # self.Reorg_ex = df['pIC50'].values

    def __len__(self):
        return len(self.smiles)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg.TOKENIZER, self.cfg.MAX_SEQ_LENGTH, self.smiles[item])
        label = np.array([])
        if not self.is_train:
            label = create_label(self.pIC50[item])
        return inputs, label

In [28]:
# 1개 미리보기
temp_train_dataset = ChemiDataset(ModelCfg, train)
temp_train_loader = DataLoader(temp_train_dataset, batch_size=1, shuffle=False, drop_last=True)
inputs, labels = next(iter(temp_train_loader))
print('inputs : ', inputs)
print('labels : ', labels.shape, labels)

inputs :  {'input_ids': tensor([[  0, 291,  63,  39,  36,  36,  44, 336,  39,  13,  39, 263,  51,  13,
          50,  63,  39,  36,  44, 336,  39, 263,  51,  13,  50,  21,  39,  63,
          39,  36,  36,  44, 336, 270, 263,  51,  13, 670,   7, 265,  22, 308,
          12, 271,  63,  39,  36,  36,  44,  65,  23, 267, 263,  51,  13,  50,
          23,  13,  71,  23, 261,  12, 271,  13,  71,  12,  39,  12,  50, 287,
          51,  13, 261, 322,  13,  39,  63,  39,  36,  44,  65,  21,  39, 263,
          51,  13,  50,  63,  39,  36,  36,  44,  65,  21, 343,  22, 268, 306,
          13,  39,  21, 293,  21,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}
lab

In [29]:
# 5개 미리보기
temp_train_dataset = ChemiDataset(ModelCfg, train)
temp_train_loader = DataLoader(temp_train_dataset, batch_size=5, shuffle=True, drop_last=True)
inputs, labels = next(iter(temp_train_loader))
print('inputs : ', inputs)
print('labels : ', labels, labels.shape)

inputs :  {'input_ids': tensor([[  0, 291,  12,  39,  13,  71,  21, 264,  12, 278,  22, 302,  12,  17,
          71,  23, 272,  24, 418,  24,  87,  23,  13,  71,  12,  50,  63,  39,
          36,  36,  44,  65,  23,  39,  63,  39,  36,  44, 336, 298, 279,  39,
          36,  36,  44, 336,  51, 279,  39,  36,  44,  65,  23,  51,  13,  82,
          22,  13, 261,  21,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1],
        [  0, 262,  12,  39, 274,  51, 279,  39,  36,  44, 336,  42,  13, 309,
         263,  51,  13,  71,  21, 308,  12,  17,  82,  22, 264,  23, 261,  12,
          39,   7,  50,  13, 308, 323,  13, 261,  21, 301,  21, 295,  21,   2,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1

# Define Model

In [30]:
class ChemoModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.MODEL_NAME, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.MODEL_NAME, config=self.config, cache_dir='./cache')
        else:
            self.model = AutoModel.from_config(self.config)

        self.fc_dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.config.hidden_size, 1)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        return last_hidden_states

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [31]:
def get_optimizer_params(model):
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    return optimizer_parameters


In [32]:
def get_scheduler(cfg, optimizer, num_train_steps):
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return scheduler

# Define Loss Class Object

In [33]:
class LoseMeter(object):
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Define function for train & valid & inference

In [34]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    losses = LoseMeter()
    epoch_display = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    epoch_display.set_description(f'Epoch [{epoch+1}/{ModelCfg.epochs}]')
    for step, (inputs, labels) in epoch_display:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        y_preds = model(inputs)
        y_preds = y_preds[::, 0]
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        epoch_display.set_postfix(train_loss=losses.avg, lr=scheduler.get_lr()[0])

    return losses.avg

def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    input_values = tqdm(test_loader, total=len(test_loader))
    for inputs, _ in input_values:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        y_preds = y_preds[::, 0]
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

# Train Loop Function

In [35]:
def train_loop(train):
    # ====================================================
    # data loader
    # ====================================================
    x_train_ = train.copy()
    train_dataset = ChemiDataset(ModelCfg, x_train_)
    train_loader = DataLoader(train_dataset, batch_size=ModelCfg.batch_size, shuffle=True, drop_last=True)

    # ====================================================
    # model
    # ====================================================
    model = ChemoModel(ModelCfg, config_path=None, pretrained=False)
    torch.save(model.config, ModelCfg.MODEL_PATH + 'config.pth')
    model.to(device)

    # ====================================================
    # optimizer
    # ====================================================
    optimizer_parameters = get_optimizer_params(model)
    optimizer = AdamW(optimizer_parameters, lr=2e-5, eps=1e-8, betas=(0.9, 0.999))

    # ====================================================
    # scheduler
    # ====================================================
    num_train_steps = int(len(x_train_) / ModelCfg.batch_size * ModelCfg.epochs)
    scheduler = get_scheduler(ModelCfg, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = MY_RMSELoss
    best_score = np.inf
    for epoch in range(ModelCfg.epochs):
        start_time = time.time()

        # train
        avg_train_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # scoring
        epoch_display = tqdm([0], leave=True)
        for _ in epoch_display:
            epoch_display.set_description(f'Epoch [{epoch+1}/{ModelCfg.epochs}]')

            score = avg_train_loss
            if best_score > score:
                best_score = score
                torch.save({'model': model.state_dict()}, f"{ModelCfg.MODEL_PATH}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth")

            elapsed = time.time() - start_time
            epoch_display.set_postfix(avg_train_loss=f'{avg_train_loss:.4f}', time=f'{elapsed/60:.0f}m', Score=f'{score:.4f}')

    torch.cuda.empty_cache()
    gc.collect()

# Training or Inference

In [36]:
# 데이타 읽기
print('read data.............')
train, test, sample_submission = read_data_list()

# tokenizer 생성
print('create tokenizer.............')
tokenizer = make_tokenizer()
ModelCfg.TOKENIZER = tokenizer

# tokenizer에 입력하는 최대 길이 계산
print('calculate max sequence length.............')
max_seq_length = cacl_max_lenth(train)
ModelCfg.MAX_SEQ_LENGTH = max_seq_length

# 훈련
print(f'TRAINING [{ModelCfg.MODEL_NAME}] MODEL.............')
train_loop(train)

# # 예측
# print(f'INFERENCE [{ModelCfg.MODEL_NAME}] MODEL.............')
# test_dataset = ChemiDataset(ModelCfg, test, True)
# test_loader = DataLoader(test_dataset, batch_size=ModelCfg.batch_size, shuffle=False, drop_last=False)
# model = ChemoModel(ModelCfg, config_path=None, pretrained=False)
# state = torch.load(f"{ModelCfg.MODEL_PATH}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth", map_location=torch.device('cpu'))
# model.load_state_dict(state['model'])
# prediction = inference_fn(test_loader, model, device)
# del model, state
# gc.collect()
# torch.cuda.empty_cache()

# print(f'[{ModelCfg.MODEL_NAME}] predictions shape : ', np.array(prediction).shape)


read data.............
train.shape:  (1952, 15)
test.shape:  (113, 2)
sample_submission.shape:  (113, 2)
create tokenizer.............
tokenizer object load & save......[seyonec/PubChem10M_SMILES_BPE_180k]
{'input_ids': [0, 315, 21, 71, 263, 51, 13, 71, 22, 71, 12, 358, 22, 39, 13, 82, 12, 39, 13, 71, 21, 33, 51, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Cn', '1', 'c', '(=', 'O', ')', 'c', '2', 'c', '(', 'ncn', '2', 'C', ')', 'n', '(', 'C', ')', 'c', '1', '=', 'O']
calculate max sequence length.............
calculate max sequence length....[seyonec/PubChem10M_SMILES_BPE_180k]


  0%|          | 0/1952 [00:00<?, ?it/s]

SMILES max length :  102
final max data length :  104
TRAINING [seyonec/PubChem10M_SMILES_BPE_180k] MODEL.............


  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
# 예측
print(f'INFERENCE [{ModelCfg.MODEL_NAME}] MODEL.............')
test_dataset = ChemiDataset(ModelCfg, test, True)
test_loader = DataLoader(test_dataset, batch_size=ModelCfg.batch_size, shuffle=False, drop_last=False)
model = ChemoModel(ModelCfg, config_path=None, pretrained=True)
state = torch.load(f"{ModelCfg.MODEL_PATH}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth", map_location=torch.device('cpu'))
model.load_state_dict(state['model'])
prediction = inference_fn(test_loader, model, device)
del model, state
gc.collect()
torch.cuda.empty_cache()

print(f'[{ModelCfg.MODEL_NAME}] predictions shape : ', np.array(prediction).shape)

INFERENCE [seyonec/PubChem10M_SMILES_BPE_180k] MODEL.............


pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

[seyonec/PubChem10M_SMILES_BPE_180k] predictions shape :  (113, 1)


# Ready for Submission

In [40]:
import pandas as pd

sample_submission = pd.read_csv('/content/drive/MyDrive/case1/open/sample_submission.csv')

sample_submission

,ID,IC50_nM
0,TEST_000,1295.44120
1,TEST_001,139.09500
2,TEST_002,147.80534
3,TEST_003,427.55118
4,TEST_004,187.08601
...,...,...
108,TEST_108,140.79654
109,TEST_109,1032.08120
110,TEST_110,118.20988
111,TEST_111,1235.23250


In [41]:
sample_submission['IC50_nM'] = pIC50_to_IC50(prediction-1)

fname = f"/content/drive/MyDrive/case1/submit/nlp_same.csv"
sample_submission.to_csv(fname, index=False)
print(f"'{fname}' saved complete.")

display(sample_submission)

'/content/drive/MyDrive/case1/submit/nlp_same.csv' saved complete.


,ID,IC50_nM
0,TEST_000,129.544118
1,TEST_001,13.909499
2,TEST_002,14.780530
3,TEST_003,42.755107
4,TEST_004,18.708588
...,...,...
108,TEST_108,14.079655
109,TEST_109,103.208152
110,TEST_110,11.820994
111,TEST_111,123.523247


# End Of Program